In [4]:
# imports
import numpy as np
import pandas as pd
import pickle
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from tqdm.auto import tqdm

In [5]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])
INDEX = 'info_clone'
NEWINDEX = 'info_clone2'
es.ping()

True

In [6]:
t = es.indices.stats(index=INDEX)
SIZE = t['_all']['primaries']['docs']['count']

In [15]:
s = Search(using=es, index=INDEX)
q = Q("wildcard", name='*')
query = s.query(q) #.extra(size=SIZE)
#response = query.execute()
#print(len(response))

In [17]:
# Find files that contain no markdown or code
def check_empty(es, nid, name):
    mkdn = es.get(index='markdown', id=nid)
    mkdnlist = mkdn['_source']['markdown']
    code = es.get(index='code', id=nid)
    codelist = code['_source']['code']
    comments = es.get(index='comments2', id=nid)
    cmtlist = comments['_source']['comments']
    if mkdnlist == [] and codelist == [] and cmtlist == []:
        return True
    else:
        return False

In [ ]:
# empty_files = []
# for i in range(len(response)):
#     nid = response[i].meta.id
#     name = response[i]['name']
#     if check_empty(es, nid, name):
#         empty_files.append((name, nid))

empty_files = []
for hit in s.scan():
    nid = hit.meta.id
    name = hit.name
    if check_empty(es, nid, name):
        empty_files.append((name, nid))

In [15]:
len(empty_files)

[('python_notebook_2017-02-19T16:27:47.271-0800.ipynb', '556437'),
 ('python_notebook_2017-03-30T17:35:51.587-0700.ipynb', '556867'),
 ('python_notebook_2017-03-24T20:17:35.130-0700.ipynb', '556801')]

In [ ]:
with open('empty.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
with open('empty.pkl', 'rb') as f:
     data = pickle.load(f)

In [ ]:
#es.delete(index=NEWINDEX, id=nid)